# Hyperparameter Tuning on Heart Failure Prediction Dataset¶

Importing required dependencies

In [8]:
import azureml.core
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, loguniform
import os, shutil

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


## Dataset Description

This project aims to leverage the power of Automated Machine Learning (AutoML) to predict mortality resulting from heart failure, a prevalent consequence of Cardiovascular diseases (CVDs). Heart failure manifests when the heart's ability to pump blood adequately to meet the body's metabolic demands diminishes, often leading to severe health complications and, in some cases, death.

The dataset chosen for this predictive task is the [Heart Failure Prediction dataset](https://raw.githubusercontent.com/robiulrafi/AZURE_ML_ND_PORTFOLIO/main/project_1/heart_failure_clinical_records_dataset.csv), which encompasses comprehensive clinical records from 299 heart failure patients. This dataset comprises a rich array of 12 features, capturing various aspects of patients' clinical conditions, physiological parameters, and lifestyle attributes.

Our primary objective revolves around training a robust binary classification model capable of predicting the DEATH_EVENT column. This binary indicator signifies whether a patient survived or succumbed to heart failure before the designated follow-up period elapsed. The predictive power of our model hinges on the information gleaned from the remaining 11 columns, serving as predictors. It's worth noting that the time feature, which may have provided temporal context, was deliberately excluded from the training process. This decision was made to ensure the model's practicality for future use, as obtaining time-based data for new patients post-deployment is often unfeasible.

The successful development and deployment of accurate prediction models hold immense potential for enhancing patient care and clinical decision-making within hospital settings. By enabling healthcare professionals to anticipate mortality risks associated with cardiovascular diseases more effectively, these models can facilitate timely interventions and personalized treatment strategies, ultimately improving patient outcomes and reducing the burden on healthcare systems.

## Initializing the Workspace

Workspace initialization from the given configuration. 

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-252373
aml-quickstarts-252373
westus2
a0a76bad-11a1-4a2d-9887-97a29122c8ed


## Creating the Capstone hyperdrive experiment on Azure ML 

The experiment to track all the submitted and completed runs in the given workspace.

In [3]:
# Choose a name for the run history container in the workspace
experiment_name = 'hyperdrive-capstone'
experiment = Experiment(ws, experiment_name)

run = experiment.start_logging()

## Adding or Creating the Compute Cluster  

Creatung the compute cluster for the AutoML run

In [4]:
# choose a name for your cluster
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
cluster_name = "notebook252373"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f'{cluster_name} exists already')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    
    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
    compute_target.wait_for_completion(show_output=True)
    
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

notebook252373 exists already
notebook252373 ComputeInstance Succeeded


## Hyperdrive Configuration

Following configuration is chosen for hypertuning experiment:

1. **Early Termination Policy**:
   - `BanditPolicy` is chosen as the early termination policy. It automatically terminates poorly performing runs based on slack criteria, making it suitable for resource-efficient experimentation.
   - `evaluation_interval`: Specifies the frequency for applying the policy. In this case, every 2 iterations (or runs) will be evaluated.
   - `delay_evaluation`: Sets the delay for the first policy evaluation. This allows sufficient runs to be completed before applying the early termination policy.
   - `slack_factor`: Defines the slack allowed with respect to the best performing run. If a run's performance falls below this slack factor compared to the best run, it will be terminated early.

2. **Parameter Sampling**:
   - `RandomParameterSampling` is used for hyperparameter sampling. It randomly selects parameter values from the specified search space, allowing for comprehensive exploration of hyperparameters.
   - For `--C`, a choice of values ranging from 0.001 to 1000 is provided.
   - For `--max_iter`, several discrete values are provided, ranging from 50 to 300.

3. **Estimator and HyperDrive Config**:
   - An environment (`env`) is created from pip requirements specified in the `requirements.txt` file. This ensures consistent dependencies across experiments.
   - `ScriptRunConfig` defines the configuration for running a script (`train.py`) on the specified compute target (`compute_target`). The environment (`env`) is also specified to ensure reproducibility.
   
4. **HyperDriveConfig**:
   - `HyperDriveConfig` encapsulates the configuration for HyperDrive, Microsoft Azure's hyperparameter tuning service.
   - `run_config`: Specifies the configuration for running the script, including the source directory, script file, compute target, and environment.
   - `hyperparameter_sampling`: Specifies the hyperparameter sampling method, which is set to `param_sampling`.
   - `policy`: Sets the early termination policy to `early_termination_policy`.
   - `primary_metric_name`: Defines the primary metric to optimize during hyperparameter tuning. In this case, it's set to `'Accuracy'`.
   - `primary_metric_goal`: Specifies whether to maximize or minimize the primary metric. Since accuracy is being maximized, `PrimaryMetricGoal.MAXIMIZE` is chosen.
   - `max_total_runs`: Sets the maximum total number of runs to execute. This limits the overall computational resources consumed by the hyperparameter tuning process.
   - `max_concurrent_runs`: Defines the maximum number of runs to execute concurrently. It helps in managing computational resources efficiently by limiting the number of concurrent training jobs.

In [5]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
# Specify a Policy
early_termination_policy = BanditPolicy(evaluation_interval=2, delay_evaluation=5, slack_factor=0.1)

param_sampling = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)

# Create your estimator and hyperdrive config
env = Environment.from_pip_requirements(name='venv', file_path='./requirements.txt')

estimator = ScriptRunConfig(
    source_directory=".",
    script='train.py',
    compute_target=compute_target,
    environment=env
    )
               
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(
                            run_config=estimator,
                            hyperparameter_sampling=param_sampling,
                            policy=early_termination_policy,
                            primary_metric_name='Accuracy',
                            primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                            max_total_runs=16,
                            max_concurrent_runs=4
                            )

In [6]:
# Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

In [7]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_daff964b-16ff-48ac-99c3-170db3a7351a
Web View: https://ml.azure.com/runs/HD_daff964b-16ff-48ac-99c3-170db3a7351a?wsid=/subscriptions/a0a76bad-11a1-4a2d-9887-97a29122c8ed/resourcegroups/aml-quickstarts-252373/workspaces/quick-starts-ws-252373&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-02-10T23:12:56.949671][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-02-10T23:12:57.3942241Z][SCHEDULER][INFO]Scheduling job, id='HD_daff964b-16ff-48ac-99c3-170db3a7351a_0' 
[2024-02-10T23:12:57.5363055Z][SCHEDULER][INFO]Scheduling job, id='HD_daff964b-16ff-48ac-99c3-170db3a7351a_1' 
[2024-02-10T23:12:57.6843539Z][SCHEDULER][INFO]Scheduling job, id='HD_daff964b-16ff-48ac-99c3-170db3a7351a_2' 
[2024-02-10T23:12:57.8874877Z][SCHEDULER][INFO]Scheduling job, id='HD_daff964b-16ff-48ac-99c3-170db3a7351a_3' 
[2024-02-10T23:12:57.741180][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_daff964b-16ff-48ac-99c3-170db3a7351a',
 'target': 'notebook252373',
 'status': 'Completed',
 'startTimeUtc': '2024-02-10T23:12:56.085101Z',
 'endTimeUtc': '2024-02-10T23:54:15.833177Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4a04a01f-47b2-4560-a8eb-7deb7545e84e',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '44',
  'best_child_run_id': 'HD_daff964b-16ff-48ac-99c3-170db3a7351a_9',
  'score': '0.75',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_daff964b-16ff-48ac-99c3-170db3a7351a_9'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azur

## Best Model

In [10]:
# get_children_sorted_by_primary_metric: Returns a list of children sorted by their best primary metric.
# Each child in the result has run id, hyperparameters, best primary metric value and status.

print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))
# get_hyperparameters
# Return the hyperparameters for all the child runs that were launched by this HyperDriveRun.

print(hyperdrive_run.get_hyperparameters())


[{'run_id': 'HD_daff964b-16ff-48ac-99c3-170db3a7351a_14', 'hyperparameters': '{"--C": 1000, "--max_iter": 200}', 'best_primary_metric': 0.75, 'status': 'Completed'}, {'run_id': 'HD_daff964b-16ff-48ac-99c3-170db3a7351a_9', 'hyperparameters': '{"--C": 500, "--max_iter": 100}', 'best_primary_metric': 0.75, 'status': 'Completed'}, {'run_id': 'HD_daff964b-16ff-48ac-99c3-170db3a7351a_10', 'hyperparameters': '{"--C": 500, "--max_iter": 200}', 'best_primary_metric': 0.75, 'status': 'Completed'}, {'run_id': 'HD_daff964b-16ff-48ac-99c3-170db3a7351a_6', 'hyperparameters': '{"--C": 100, "--max_iter": 200}', 'best_primary_metric': 0.75, 'status': 'Completed'}, {'run_id': 'HD_daff964b-16ff-48ac-99c3-170db3a7351a_15', 'hyperparameters': '{"--C": 50, "--max_iter": 200}', 'best_primary_metric': 0.7333333333333333, 'status': 'Completed'}, {'run_id': 'HD_daff964b-16ff-48ac-99c3-170db3a7351a_13', 'hyperparameters': '{"--C": 0.01, "--max_iter": 300}', 'best_primary_metric': 0.7333333333333333, 'status': 'C

In [11]:

# get_best_run_by_primary_metric()
# Returns the best Run, or None if no child has the primary metric.
best_run_hyp = hyperdrive_run.get_best_run_by_primary_metric()

# get_metrics()
# Returns the metrics from all the runs that were launched by this HyperDriveRun.
print("Best run metrics :",best_run_hyp.get_metrics())


# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run_hyp.get_details())
print('===================================================')

# get_file_names()
# Returns a list of the files that are stored in association with the run.

print("Best run file names :",best_run_hyp.get_file_names())
print('===================================================')

Best run metrics : {'Regularization Strength:': 500.0, 'Max iterations:': 100, 'Accuracy': 0.75}
Best run details : {'runId': 'HD_daff964b-16ff-48ac-99c3-170db3a7351a_9', 'target': 'notebook252373', 'status': 'Completed', 'startTimeUtc': '2024-02-10T23:39:42.856674Z', 'endTimeUtc': '2024-02-10T23:43:02.181433Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlcdsi', '_azureml.ClusterName': 'notebook252373', 'ContentSnapshotId': '4a04a01f-47b2-4560-a8eb-7deb7545e84e', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '500', '--max_iter', '100'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'notebook252373', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 25920

In [12]:
hyperdrive_run.get_best_run_by_primary_metric()

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive-capstone,HD_daff964b-16ff-48ac-99c3-170db3a7351a_9,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [21]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:', best_run_metrics['Regularization Strength:'])
print('\n Maximum Iteration:', best_run_metrics['Max iterations:'])

Best Run Id:  HD_daff964b-16ff-48ac-99c3-170db3a7351a_9

 Accuracy: 0.75

 Regularization Strength: 500.0

 Maximum Iteration: 100


In [15]:
print(best_run.get_file_names())

['logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [17]:
# Register the model
best_run.register_model(model_path='outputs/', model_name='hyperdrive-best-model',
                   tags={'Training context':'Parameterized SKLearn Estimator', 'type': 'Classification'},
                   properties={'Accuracy': best_run_metrics['Accuracy']},
                   description = 'Hyperparameter based Heart Failure Predictor')

Model(workspace=Workspace.create(name='quick-starts-ws-252373', subscription_id='a0a76bad-11a1-4a2d-9887-97a29122c8ed', resource_group='aml-quickstarts-252373'), name=hyperdrive-best-model, id=hyperdrive-best-model:1, version=1, tags={'Training context': 'Parameterized SKLearn Estimator', 'type': 'Classification'}, properties={'Accuracy': '0.75'})

## Model Deployment

In the deployment phase, only one of the two trained models will be deployed. Considering that the voting ensemble method of AutoML configuration provides better primary metrics than the hyperdrive case, the model associated with that is being considered for deployment.

In [11]:
compute_target.delete()